In [1]:
import snowflake.connector
import pandas as pd
from datasets import Dataset, ClassLabel
from dotenv import load_dotenv
import os
import wandb

/Users/shri/opt/anaconda3/envs/hackathon/lib/python3.10/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
load_dotenv()

True

In [3]:
conn = snowflake.connector.connect(
    user=os.environ["SF_USER"],
    password=os.environ["SF_PWD"],
    account=os.environ["SF_ACCOUNT"],
    database=os.environ["SF_DB"],
    schema=os.environ["SF_SCHEMA"]
)

In [4]:
# query the data from Snowflake and create a Pandas dataframe
query = 'SELECT INTENT, UTT, PARTITION, SCENARIO FROM ALEXA_MASSIVE_INTENTS_RAW;' #query = 'SELECT * FROM intent_dataset;'
alexa_df = pd.read_sql(query, conn)

/var/folders/k7/kvt_fn2n5kj_cccj1v8w85fc0000gn/T/ipykernel_11140/1906349022.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  alexa_df = pd.read_sql(query, conn)


In [5]:
alexa_df

,INTENT,UTT,PARTITION,SCENARIO
0,alarm_set,wake me up at five am this week,test,alarm
1,alarm_set,wake me up at nine am on friday,train,alarm
2,alarm_set,set an alarm for two hours from now,train,alarm
3,audio_volume_mute,quiet,test,audio
4,audio_volume_mute,olly quiet,train,audio
...,...,...,...,...
16516,email_query,do i have emails,train,email
16517,email_query,what emails are new,train,email
16518,email_query,do i have new emails from john,train,email
16519,email_query,has john sent me an email,test,email


In [6]:
alexa_df.drop(["SCENARIO"], axis=1, inplace=True)

In [7]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from datasets import Features, Value

In [8]:
# get the list of unique labels in the 'category' column
label_list = alexa_df['INTENT'].unique().tolist()

# instantiate a ClassLabel object with the number of classes and the names of the labels
num_classes = len(label_list)

In [9]:
# create Hugging Face dataset
features = Features({"INTENT": ClassLabel(num_classes=num_classes, names=label_list),
                     "UTT": Value("string"), 
                     "PARTITION": Value("string"),})
dataset = Dataset.from_pandas(alexa_df, features=features)


In [10]:
dataset = dataset.rename_column("INTENT", "label")

In [11]:
# split dataset into train and test sets based on the value of "SOURCE"
train_dataset = dataset.filter(lambda example: example["PARTITION"] == "train")
dev_dataset = dataset.filter(lambda example: example["PARTITION"] == "dev")
test_dataset = dataset.filter(lambda example: example["PARTITION"] == "test")

Filter:   0%|          | 0/16521 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16521 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16521 [00:00<?, ? examples/s]

In [12]:
train_dataset

Dataset({
    features: ['label', 'UTT', 'PARTITION'],
    num_rows: 11514
})

In [14]:
max_train_samples = 500
trunc_train_dataset = train_dataset.shuffle().select(range(max_train_samples))

In [15]:
trunc_train_dataset

Dataset({
    features: ['label', 'UTT', 'PARTITION'],
    num_rows: 500
})

In [16]:
dev_dataset

Dataset({
    features: ['label', 'UTT', 'PARTITION'],
    num_rows: 2033
})

In [17]:
max_dev_samples = 90
trunc_dev_dataset = dev_dataset.shuffle().select(range(max_dev_samples))

In [18]:
trunc_dev_dataset

Dataset({
    features: ['label', 'UTT', 'PARTITION'],
    num_rows: 90
})

In [19]:
test_dataset

Dataset({
    features: ['label', 'UTT', 'PARTITION'],
    num_rows: 2974
})

In [20]:
max_test_samples = 130
trunc_test_dataset = test_dataset.shuffle().select(range(max_test_samples))
trunc_test_dataset

Dataset({
    features: ['label', 'UTT', 'PARTITION'],
    num_rows: 130
})

In [21]:
model_name = "joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english"

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# load the pre-trained BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['UTT'], padding="max_length", truncation=True)

trunc_train_dataset = trunc_train_dataset.map(tokenize_function, batched=True)
trunc_dev_dataset = trunc_dev_dataset.map(tokenize_function, batched=True)
trunc_test_dataset = trunc_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

# Train Model

In [24]:
# load the pre-trained small BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

In [25]:
from datasets import load_metric
metric = load_metric("accuracy")
import numpy as np
def compute_metrics(preds):
    logits, labels = preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/var/folders/k7/kvt_fn2n5kj_cccj1v8w85fc0000gn/T/ipykernel_11140/1996331686.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [26]:
# set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',     # evaluate model after every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=32,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=30,              # number of training epochs
    weight_decay=0.01,               # weight decay
    push_to_hub=False,               # whether to upload the model checkpoint to the Hub
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1,
    report_to=None #"wandb"
)

# create the Trainer object
trainer = Trainer(
    model=model,                         # the instantiated model to be trained
    args=training_args,                  # training arguments
    train_dataset=trunc_train_dataset,         # the training dataset
    eval_dataset=trunc_dev_dataset,            # the evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
# start the training
trainer.train()

/Users/shri/opt/anaconda3/envs/hackathon/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: shri-adke (loyalhealth). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 3.6437575817108154, 'eval_accuracy': 0.24, 'eval_runtime': 6.7409, 'eval_samples_per_second': 3.709, 'eval_steps_per_second': 0.148, 'epoch': 9.0}
{'loss': 1.3398, 'learning_rate': 1.7142857142857145e-06, 'epoch': 9.14}
{'loss': 2.5963, 'learning_rate': 1.4285714285714286e-06, 'epoch': 9.29}
{'loss': 2.5325, 'learning_rate': 1.142857142857143e-06, 'epoch': 9.43}
{'loss': 2.1737, 'learning_rate': 8.571428571428572e-07, 'epoch': 9.57}
{'loss': 2.4386, 'learning_rate': 5.714285714285715e-07, 'epoch': 9.71}
{'loss': 2.1589, 'learning_rate': 2.8571428571428575e-07, 'epoch': 9.86}
{'loss': 2.0316, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 3.6343860626220703, 'eval_accuracy': 0.24, 'eval_runtime': 6.7288, 'eval_samples_per_second': 3.715, 'eval_steps_per_second': 0.149, 'epoch': 10.0}
{'train_runtime': 936.487, 'train_samples_per_second': 1.068, 'train_steps_per_second': 0.075, 'train_loss': 3.6161604336329867, 'epoch': 10.0}


TrainOutput(global_step=70, training_loss=3.6161604336329867, metrics={'train_runtime': 936.487, 'train_samples_per_second': 1.068, 'train_steps_per_second': 0.075, 'train_loss': 3.6161604336329867, 'epoch': 10.0})

In [32]:
# To visualize metrics on test dataset
trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=trunc_train_dataset,
    eval_dataset=trunc_test_dataset,
    compute_metrics=compute_metrics,
)
trainer2.evaluate()

In [ ]:
# Store the model
import tarfile
save_path = "./results/"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

with tarfile.open("./results/model.tar.gz", mode='w:gz') as archive:
    archive.add(save_path, arcname='.')